In [ ]:
# make local editable packages automatically reload
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import nd2
# Import dependencies
import numpy as np
from cellpose_omni import models, core

# This checks to see if you have set up your GPU properly.
# CPU performance is a lot slower, but not a problem if you 
# are only processing a few images.
use_GPU = core.use_gpu()
print('>>> GPU activated? %d'%use_GPU)

# for plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi'] = 300
plt.style.use('dark_background')
%matplotlib inline

In [ ]:
from cellpose_omni import io, transforms
from omnipose.utils import normalize99

In [ ]:
nd2_file = nd2.imread("../real_data/agar_pads/40x_SB6_dilute001.nd2")

In [ ]:
imgs = [x[0] for x in nd2_file]

In [ ]:
nimg = len(imgs)

fig = plt.figure(figsize=[40]*2,frameon=False) # initialize figure
for k in range(len(imgs)):
    img = transforms.move_min_dim(imgs[k]) # move the channel dimension last
    if len(img.shape)>2:
        # imgs[k] = img[:,:,1] # could pick out a specific channel
        imgs[k] = np.mean(img,axis=-1) # or just turn into grayscale 
        
    imgs[k] = normalize99(imgs[k])
    # imgs[k] = np.pad(imgs[k],10,'edge')

In [ ]:
from cellpose_omni import models
from cellpose_omni.models import MODEL_NAMES
model_name = 'bact_phase_omni'
model = models.CellposeModel(gpu=use_GPU, model_type=model_name)

In [ ]:
chans = [0,0] #this means segment based on first channel, no second channel 

n = [-3] # make a list of integers to select which images you want to segment
n = range(nimg) # or just segment them all 

# define parameters
mask_threshold = -1
verbose = 0 # turn on if you want to see more output 
use_gpu = use_GPU #defined above
transparency = True # transparency in flow output
rescale=None # give this a number if you need to upscale or downscale your images
omni = True # we can turn off Omnipose mask reconstruction, not advised 
flow_threshold = 0 # default is .4, but only needed if there are spurious masks to clean up; slows down output
niter = None # None lets Omnipose calculate # of Euler iterations (usually <20) but you can tune it for over/under segmentation 
resample = True #whether or not to run dynamics on rescaled grid or original grid 
cluster = True # use DBSCAN clustering
augment = False # average the outputs from flipped (augmented) images; slower, usually not needed 
tile = False # break up image into smaller parts then stitch together
affinity_seg = 1 #new feature, stay tuned...

masks, flows, styles = model.eval([imgs[i] for i in n],
                                  channels=chans,
                                  rescale=rescale,
                                  mask_threshold=mask_threshold,
                                  transparency=transparency,
                                  flow_threshold=flow_threshold,
                                  niter=niter,
                                  omni=omni,
                                  cluster=cluster, 
                                  resample=resample,
                                  verbose=verbose, 
                                  affinity_seg=affinity_seg,
                                  tile=tile,
                                  augment=augment)

In [ ]:
from cellpose_omni import plot
import omnipose

for idx,i in enumerate(n):
    maski = masks[idx] # get masks
    bdi = flows[idx][-1] # get boundaries
    flowi = flows[idx][0] # get RGB flows 
    fig = plt.figure(figsize=(12,5),frameon=False)
    plot.show_segmentation(fig, omnipose.utils.normalize99(imgs[i]), 
                           maski, flowi, bdi, channels=chans, omni=True, bg_color=0)
    plt.tight_layout()
    plt.show()
    plt.close()

In [ ]:
from PIL import Image
try:
    os.mkdir("../real_data/40x_SB6_dilute001_segmented")
except:
    pass

[Image.fromarray(masks[x]).save(f"../real_data/40x_SB6_dilute001_segmented/{str(x).zfill(3)}.png") for x in range(len(imgs))]